# "Zillow Zing Zestimate Project"

## Background

*Zillow is an online real estate database company founded in 2006 - Wikipedia.* “Zestimates” are estimated home values based on 7.5 million statistical and machine learning models that analyze hundreds of data points on each property. And, by continually improving the median margin of error (from 14% at the onset to 5% today). The Zillow Zestimate Zing Project will predict the error between Zestimate and the sales price of a home. 

- Project By Ednalyn C. De Dios
- Codeup Data Science
- ADA Cohort
- 25 March 2019

## Project Planning

### Goal

- Develop a linear regression model that will best predict the log error of the Zestimate.

### Deliverables

1. Report
1. Jupyter Notebook

### Data Dictionary & Details


***We will be using 2016 data.***

- logerror: 
- bathroomcnt: 
- bedroomcnt: 
- calculatedfinishedsquarefeet: 
- fullbathcnt: 
- garagecarcnt: 
- roomcnt: 
- yearbuilt: 
- taxvalueddollarcnt: 
- taxamount: 
- regionidcounty: 
- regionidcity: 
- regionidzip: 
- regionidneighborhood: 

### Initial Thoughts, Ideas, Hypotheses

- In addition to the given columns variables), I think that location-related variables also ha e a role to play in driving the log error.

### Prepare the Environment

Libraries imported below for data wrangling, exploring, visualizing, and modeling will be used throughout the project.

In [1]:
# Wrangling
import pandas as pd
import math

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Modeling
import statsmodels.api as sm

from scipy.stats import pearsonr

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.feature_selection import f_regression

from env import user, host, password, path

#### Utilities
*Functions to export in the `util.py` later.*

In [2]:
def summarize_df(df):
    print("Summary of Data\n")
    print("Rows & Columns:\n")
    print(df.shape)
    print("Variables:\n")
    var_names = df.columns.values
    print(var_names)
    print("\nColumn Info:\n")
    print(df.info())
    print("\nNumeric Summary Stats:\n")
    print(df.describe())
    print("\nTop 5 Values:\n")
    for var in var_names:
        print(var+":")
        print(df[var].value_counts().head())
        print('\n')
    print("\nMissing Values:\n")
    null_counts = df.isnull().sum()
    if len(null_counts[null_counts > 0]) == 0:
        print("No missing values")
    else:
        print(null_counts[null_counts > 0])
    print("\nFirst 5 rows:\n")
    print(df.head())
    print("\nEnd of Summary")

### Data Acquisition

#### Acquire via csv
- sample until you have the right query (and then pull entire dataset)
- export to local csv 
- read from local csv (sample, but larger sample than your first) 
- save sql query to add into to_sql() function in python. 


In [3]:
# Acquire the data via csv
train_df = pd.read_csv("train_2016_v2.csv", low_memory=False)
properties_df = pd.read_csv("properties_2016.csv", low_memory=False)

# Generate a cohesive dataset
df = pd.merge(train_df, properties_df, on='parcelid', how='inner')
df = df[['parcelid',
         'logerror',
         'bathroomcnt',
         'calculatedfinishedsquarefeet',
         'fullbathcnt',
         'garagecarcnt',
         'roomcnt',
         'yearbuilt',
         'taxvaluedollarcnt',
         'taxamount',
         'regionidzip',
         'regionidneighborhood'
        ]]

In [4]:
# Log errors were filtered organically after merging
print(df.isnull().sum())
df.shape

parcelid                            0
logerror                            0
bathroomcnt                         0
calculatedfinishedsquarefeet      661
fullbathcnt                      1182
garagecarcnt                    60338
roomcnt                             0
yearbuilt                         756
taxvaluedollarcnt                   1
taxamount                           6
regionidzip                        35
regionidneighborhood            54263
dtype: int64


(90275, 12)

In [5]:
# Remove these columns entirely due to high number of null values
df = df.drop(['regionidneighborhood', 'garagecarcnt'], axis=1)
df

,parcelid,logerror,bathroomcnt,calculatedfinishedsquarefeet,fullbathcnt,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,regionidzip
0,11016594,0.0276,2.0,1684.0,2.0,0.0,1959.0,360170.0,6735.88,96370.0
1,14366692,-0.1684,3.5,2263.0,3.0,0.0,2014.0,585529.0,10153.02,96962.0
2,12098116,-0.0040,3.0,2217.0,3.0,0.0,1940.0,119906.0,11484.48,96293.0
3,12643413,0.0218,2.0,839.0,2.0,0.0,1987.0,244880.0,3048.74,96222.0
4,14432541,-0.0050,2.5,2283.0,2.0,8.0,1981.0,434551.0,5488.96,96961.0
5,11509835,-0.2705,4.0,3067.0,4.0,0.0,1982.0,2447951.0,27126.57,96109.0
6,12286022,0.0440,1.0,1297.0,1.0,0.0,1939.0,111521.0,2304.97,96091.0
7,17177301,0.1638,2.5,1763.0,2.0,6.0,1994.0,306000.0,3745.50,97101.0
8,14739064,-0.0030,1.0,796.0,1.0,0.0,1984.0,210064.0,2172.88,96987.0
9,14677559,0.0843,2.0,1260.0,2.0,5.0,1977.0,190960.0,1940.26,96963.0


#### Summarize the data. 

In [6]:
summarize_df(df)

Summary of Data

Rows & Columns:

(90275, 10)
Variables:

['parcelid' 'logerror' 'bathroomcnt' 'calculatedfinishedsquarefeet'
 'fullbathcnt' 'roomcnt' 'yearbuilt' 'taxvaluedollarcnt' 'taxamount'
 'regionidzip']

Column Info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90275 entries, 0 to 90274
Data columns (total 10 columns):
parcelid                        90275 non-null int64
logerror                        90275 non-null float64
bathroomcnt                     90275 non-null float64
calculatedfinishedsquarefeet    89614 non-null float64
fullbathcnt                     89093 non-null float64
roomcnt                         90275 non-null float64
yearbuilt                       89519 non-null float64
taxvaluedollarcnt               90274 non-null float64
taxamount                       90269 non-null float64
regionidzip                     90240 non-null float64
dtypes: float64(9), int64(1)
memory usage: 7.6 MB
None

Numeric Summary Stats:

           parcelid      logerror   b

### Data Preparation

Goal: leave this section with a dataset that is ready to be analyzed. 

In [7]:
# Remove rows where calculatedfinishedsquarefeet are null
df = df[df['calculatedfinishedsquarefeet'].notnull()]

# Remove rows where fullbathcnt containerb uilt  null
df = df[df['fullbathcnt'].notnull()]

# Remove rows where yearbuilt contains null
df = df[df['yearbuilt'].notnull()]

# Remove rows where taxvaluedollarcnt is null
df = df[df['taxvaluedollarcnt'].notnull()]

# Remove rows where taxamount is null
df = df[df['taxamount'].notnull()]

# Remove rows where taxvaluedollarcnt is null
df = df[df['regionidzip'].notnull()]

print(df.isnull().sum())
df

parcelid                        0
logerror                        0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
fullbathcnt                     0
roomcnt                         0
yearbuilt                       0
taxvaluedollarcnt               0
taxamount                       0
regionidzip                     0
dtype: int64


,parcelid,logerror,bathroomcnt,calculatedfinishedsquarefeet,fullbathcnt,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,regionidzip
0,11016594,0.0276,2.0,1684.0,2.0,0.0,1959.0,360170.0,6735.88,96370.0
1,14366692,-0.1684,3.5,2263.0,3.0,0.0,2014.0,585529.0,10153.02,96962.0
2,12098116,-0.0040,3.0,2217.0,3.0,0.0,1940.0,119906.0,11484.48,96293.0
3,12643413,0.0218,2.0,839.0,2.0,0.0,1987.0,244880.0,3048.74,96222.0
4,14432541,-0.0050,2.5,2283.0,2.0,8.0,1981.0,434551.0,5488.96,96961.0
5,11509835,-0.2705,4.0,3067.0,4.0,0.0,1982.0,2447951.0,27126.57,96109.0
6,12286022,0.0440,1.0,1297.0,1.0,0.0,1939.0,111521.0,2304.97,96091.0
7,17177301,0.1638,2.5,1763.0,2.0,6.0,1994.0,306000.0,3745.50,97101.0
8,14739064,-0.0030,1.0,796.0,1.0,0.0,1984.0,210064.0,2172.88,96987.0
9,14677559,0.0843,2.0,1260.0,2.0,5.0,1977.0,190960.0,1940.26,96963.0


In [14]:
df.rename(index=str, columns={"parcelid": "parcel_id",
                              "logerror": "log_error",
                              "bathroomcnt": "bathroom_count",
                              "calculatedfinishedsquarefeet": "square_feet",
                              "fullbathcnt": "full_bath_count",
                              "roomcnt": "room_count",
                              "yearbuilt": "year_built",
                              "taxvaluedollarcnt": "tax_count",
                              "taxamount": "tax_amount",
                              "regionidzip": "region_zip"
                             })

,parcel_id,log_error,bathroom_count,square_feet,full_bath_count,room_count,year_built,tax_count,tax_amount,region_zip
0,11016594,0.0276,2.0,1684.0,2.0,0.0,1959.0,360170.0,6735.88,96370.0
1,14366692,-0.1684,3.5,2263.0,3.0,0.0,2014.0,585529.0,10153.02,96962.0
2,12098116,-0.0040,3.0,2217.0,3.0,0.0,1940.0,119906.0,11484.48,96293.0
3,12643413,0.0218,2.0,839.0,2.0,0.0,1987.0,244880.0,3048.74,96222.0
4,14432541,-0.0050,2.5,2283.0,2.0,8.0,1981.0,434551.0,5488.96,96961.0
5,11509835,-0.2705,4.0,3067.0,4.0,0.0,1982.0,2447951.0,27126.57,96109.0
6,12286022,0.0440,1.0,1297.0,1.0,0.0,1939.0,111521.0,2304.97,96091.0
7,17177301,0.1638,2.5,1763.0,2.0,6.0,1994.0,306000.0,3745.50,97101.0
8,14739064,-0.0030,1.0,796.0,1.0,0.0,1984.0,210064.0,2172.88,96987.0
9,14677559,0.0843,2.0,1260.0,2.0,5.0,1977.0,190960.0,1940.26,96963.0


#### Create a variable, `colnames`, that is a list of the column names. 
Why? You will likely reference this variable later.

In [18]:
colnames = ['parcelid',
            'logerror',
            'bathroomcnt',
            'calculatedfinishedsquarefeet',
            'fullbathcnt',
            'roomcnt',
            'yearbuilt',
            'taxvaluedollarcnt',
            'taxamount',
            'regionidzip'
           ]
df

,parcelid,logerror,bathroomcnt,calculatedfinishedsquarefeet,fullbathcnt,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,regionidzip
0,11016594,0.0276,2.0,1684.0,2.0,0.0,1959.0,360170.0,6735.88,96370.0
1,14366692,-0.1684,3.5,2263.0,3.0,0.0,2014.0,585529.0,10153.02,96962.0
2,12098116,-0.0040,3.0,2217.0,3.0,0.0,1940.0,119906.0,11484.48,96293.0
3,12643413,0.0218,2.0,839.0,2.0,0.0,1987.0,244880.0,3048.74,96222.0
4,14432541,-0.0050,2.5,2283.0,2.0,8.0,1981.0,434551.0,5488.96,96961.0
5,11509835,-0.2705,4.0,3067.0,4.0,0.0,1982.0,2447951.0,27126.57,96109.0
6,12286022,0.0440,1.0,1297.0,1.0,0.0,1939.0,111521.0,2304.97,96091.0
7,17177301,0.1638,2.5,1763.0,2.0,6.0,1994.0,306000.0,3745.50,97101.0
8,14739064,-0.0030,1.0,796.0,1.0,0.0,1984.0,210064.0,2172.88,96987.0
9,14677559,0.0843,2.0,1260.0,2.0,5.0,1977.0,190960.0,1940.26,96963.0


#### Identify the data types of each variable. Why? 
You need to make sure they are what makes sense for the data and the meaning of the data that lies in that variable. If it does not, make necessary changes.

,parcelid,logerror,bathroomcnt,calculatedfinishedsquarefeet,fullbathcnt,roomcnt,yearbuilt,taxvaluedollarcnt,taxamount,regionidzip
0,11016594,0.0276,2.0,1684.0,2.0,0.0,1959.0,360170.0,6735.88,96370.0
1,14366692,-0.1684,3.5,2263.0,3.0,0.0,2014.0,585529.0,10153.02,96962.0
2,12098116,-0.0040,3.0,2217.0,3.0,0.0,1940.0,119906.0,11484.48,96293.0
3,12643413,0.0218,2.0,839.0,2.0,0.0,1987.0,244880.0,3048.74,96222.0
4,14432541,-0.0050,2.5,2283.0,2.0,8.0,1981.0,434551.0,5488.96,96961.0
5,11509835,-0.2705,4.0,3067.0,4.0,0.0,1982.0,2447951.0,27126.57,96109.0
6,12286022,0.0440,1.0,1297.0,1.0,0.0,1939.0,111521.0,2304.97,96091.0
7,17177301,0.1638,2.5,1763.0,2.0,6.0,1994.0,306000.0,3745.50,97101.0
8,14739064,-0.0030,1.0,796.0,1.0,0.0,1984.0,210064.0,2172.88,96987.0
9,14677559,0.0843,2.0,1260.0,2.0,5.0,1977.0,190960.0,1940.26,96963.0


#### Identify the columns that have missing values and the number of missing values in each column. 
Why? Missing values are going to cause issues down the line so you will need to handle those appropriately. For each variable with missing values, if it makes sense to replace those missing with a 0, do so. For those where that doesn't make sense, decide if you should drop the entire observations (rows) that contain the missing values, or drop the entire variable (column) that contains the missing values.

#### Create a list of the independent variable names (aka attributes) and assign it to `x_vars` 
Why? During exploration, you will likely use this list to refer to the attribute names.

#### Clearly identify your dependent (target) variable. 
- What is the name of the variable? 
- Is it discrete or continuous?

#### View the distribution of the numeric variables
- plot a histogram and box plot of each variable. 
- Why? To see the distribution, skewness, outliers, and unit scales. You will use this information in your decision of whether to normalize, standardize or neither.

#### BONUS:  Data standardization 
Create a new data frame that is the min-max normalization of the independent variable in the original data frame (+ the original dependent variable). You will normalize each of the numeric independent variables independently, i.e. using the min and max of each variable, not the min/max of the whole dataframe. Why? Regression is very sensitive to difference in units. It will be almost impossible to extract a meaningful linear regression model with such extreme differences in scale. For more context, see: https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc
You will use this dataframe in the future analysis.

### Data Exploration

Goal is to address each of the questions you posed in your planning & brainstorming through visual or statistical analysis.

When you have completed this step, you will have the findings from your analysis that will be used in your final report, the answers to your questions and your customer's questions that will reach the goal. 

#### Split data into training and test datasets

#### Create a jointplot for each independent variable (normalized version) with the dependent variable. 

Be sure you have Pearson's r and p-value annotated on each plot.

#### Create a feature plot using seaborn's PairGrid() of the interaction between each variable (dependent + independent). 

You may want to use a normalized dataframe (if you did that) or adjust the axis scale (set to logarithmic, e.g) so you can more clearly view the interactions.

#### Create a heatmap of the correlation between each variable pair.

#### Summarize your conclusions from all of these steps.  

#### Is the logerror significantly different for homes with 3 bedrooms vs those with 5 or more bedrooms? 
Run a t-test to test this difference.

#### Do the same for another 2 samples you are interested in comparing (e.g. those with 1 bath vs. x baths)

### Data Modeling

#### Feature Engineering & Selection

#### Are there new features you could create based on existing features that might be helpful? 

Come up with at least one possible new feature that is a calculation from 2+ existing variables. 

#### Use statsmodels ordinary least squares to assess the importance of each feature with respect to the target

#### Summarize your conclusions and next steps from your analysis in above. 
What will you try when developing your model? (which features to use/not use/etc)

#### Train & Test Model

#### Fit, predict (in-sample) & evaluate multiple linear regression models to find the best one.

- Make any changes as necessary to improve your model.

- Identify the best model after all training

- Compare the models by plotting the predictions of each model with actual values (see plot right above section 12 in 'Regression in Python' lesson

(the predictions are a 1 x 83 two dimensional matrix, but we want just a single array of predictions. We can use the .ravel method to achieve this.)

`pd.DataFrame({'actual': y_train.final_grade,  
              'lm1': y_pred_lm1.ravel(),  
              'lm2': y_pred_lm2.ravel()})\  
    .melt(id_vars=['actual'], var_name='model', value_name='prediction')\  
    .pipe((sns.relplot, 'data'), x='actual', y='prediction', hue='model')  

plt.plot([60, 100], [60, 100], c='black', ls=':')  
plt.ylim(60, 100)  
plt.xlim(60, 100)  
plt.title('Predicted vs Actual Final Grade')`  

#### Predict & evaluate on out-of-sample data (test)    

- Plot the residuals from your out-of-sample predictions.

- Summarize your expectations about how you estimate this model will perform in production.